# Extending the LUVOIR simulator to other global and local modes

In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import hcipy

from pastis.e2e_simulators.luvoir_imaging import LuvoirAPLC
from pastis.config import CONFIG_PASTIS
import pastis.util

First doing some general cleanup in `LuvoirAPLC()` so I need to make sure it still works ok.

In [ ]:
# Instantiate LUVOIR simulator
sampling = 4
apodizer_design = 'small'
optics_input = os.path.join(pastis.util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
luvoir = LuvoirAPLC(optics_input, apodizer_design, sampling)

In [ ]:
coro_im, ref_norm = luvoir.calc_psf(display_intermediate=True, ref=True)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(coro_im.shaped/ref_norm.max(), cmap='inferno', norm=LogNorm(vmin=1e-14, vmax=1e-3))
plt.colorbar()

Comment out one of the two cells below, depending on which SegmentedMirror implementation you are using.

In [ ]:
# For old SegmentedMirror (indexed aperture)
#luvoir.set_segment(18, 20e-9, 0, 0)
#luvoir.set_segment(75, 20e-9, 0, 0)

In [ ]:
# Create multi mode segmented mirror
# !! THIS CELL TAKES QUITE A WHILE TO RUN !!
n_modes_segs = 5
luvoir.create_segmented_mirror(n_modes_segs)

In [ ]:
# For usage with multi mode segmented mirror
new_command = np.zeros(120*n_modes_segs)
new_command[4] = 2e-9
new_command[53] = 2e-9
new_command[346] = 2e-9
luvoir.sm.actuators = new_command
coro_im_aber1 = luvoir.calc_psf(display_intermediate=True, ref=False)

In [ ]:
# Create low-order (Zernike) mode mirror
n_modes_zernikes = 8
luvoir.create_global_zernike_mirror(n_modes_zernikes)

In [ ]:
# For usage with Zernike mode mirror
new_command = np.zeros(n_modes_zernikes)
new_command[7] = 2e-9
luvoir.zernike_mirror.actuators = new_command
coro_im_aber1 = luvoir.calc_psf(display_intermediate=True, ref=False)

In [ ]:
# Create a good ol' continuous DM
n_acts_across = 15 
luvoir.create_continuous_deformable_mirror(n_acts_across)

In [ ]:
# For usage with continuous deformable mirror
new_command = np.zeros(n_acts_across*n_acts_across)
new_command[66] = 2e-9
new_command[77] = 2e-9
new_command[147] = 2e-9
luvoir.dm.actuators = new_command
coro_im_aber1 = luvoir.calc_psf(display_intermediate=True, ref=False)

In [ ]:
# Create high-order (ripple) mode mirror
n_ripples = 5    # need to use odd number
luvoir.create_ripple_mirror(n_ripples)

In [ ]:
# For usage with Zernike mode mirror
new_command = np.zeros(n_ripples*n_ripples)
new_command[12] = 2e-9
luvoir.ripple_mirror.actuators = new_command
coro_im_aber1 = luvoir.calc_psf(display_intermediate=True, ref=False)

In [ ]:
# Create segmented Harris mode mirror
fpath = '/Users/ilaginja/repos/PASTIS/Sensitivities2.xlsx'    # path to Harris spreadsheet
pad_orientations = np.pi / 2 * np.ones(120)
luvoir.create_segmented_harris_mirror(fpath, pad_orientations)

In [ ]:
# For usage with segmented Harris mode mirror
new_command = np.zeros(luvoir.harris_sm.num_actuators)
print(new_command.shape)
new_command[18] = 1e-9
new_command[37] = 2e-9
luvoir.harris_sm.actuators = new_command
coro_im_aber1 = luvoir.calc_psf(display_intermediate=True, ref=False)